Plot Tide Forecasts
===================

Plots the daily tidal displacements for a given location

OTIS format tidal solutions provided by Ohio State University and ESR  
- http://volkov.oce.orst.edu/tides/region.html  
- https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/
- ftp://ftp.esr.org/pub/datasets/tmd/  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  

Finite Element Solution (FES) provided by AVISO  
- https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html

#### Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/)  
 - [matplotlib: Python 2D plotting library](https://matplotlib.org/)  
 - [ipyleaflet: Jupyter / Leaflet bridge enabling interactive maps](https://github.com/jupyter-widgets/ipyleaflet)  

#### Program Dependencies

- `calc_astrol_longitudes.py`: computes the basic astronomical mean longitudes  
- `calc_delta_time.py`: calculates difference between universal and dynamic time  
- `convert_ll_xy.py`: convert lat/lon points to and from projected coordinates  
- `load_constituent.py`: loads parameters for a given tidal constituent  
- `load_nodal_corrections.py`: load the nodal corrections for tidal constituents  
- `infer_minor_corrections.py`: return corrections for minor constituents  
- `model.py`: retrieves tide model parameters for named tide models  
- `read_tide_model.py`: extract tidal harmonic constants from OTIS tide models  
- `read_netcdf_model.py`: extract tidal harmonic constants from netcdf models  
- `read_GOT_model.py`: extract tidal harmonic constants from GSFC GOT models  
- `read_FES_model.py`: extract tidal harmonic constants from FES tide models  
- `predict_tidal_ts.py`: predict tidal time series at a location using harmonic constants  

This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.  
The widgets can be installed as described below.  
```
pip3 install --user ipywidgets
jupyter nbextension install --user --py widgetsnbextension
jupyter nbextension enable --user --py widgetsnbextension
jupyter-notebook
```

#### Load modules

In [ ]:
from __future__ import print_function

import os
import datetime
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import ipyleaflet as leaflet

import pyTMD.time
import pyTMD.model
from pyTMD.calc_delta_time import calc_delta_time
from pyTMD.infer_minor_corrections import infer_minor_corrections
from pyTMD.predict_tidal_ts import predict_tidal_ts
from pyTMD.read_tide_model import extract_tidal_constants
from pyTMD.read_netcdf_model import extract_netcdf_constants
from pyTMD.read_GOT_model import extract_GOT_constants
from pyTMD.read_FES_model import extract_FES_constants
from pyTMD.spatial import wrap_longitudes
# autoreload
%load_ext autoreload
%autoreload 2

In [ ]:
# set the directory with tide models
dirText = widgets.Text(
    value=os.getcwd(),
    description='Directory:',
    disabled=False
)

# dropdown menu for setting tide model
model_list = ['CATS0201','CATS2008','TPXO9-atlas','TPXO9-atlas-v2',
    'TPXO9-atlas-v3','TPXO9-atlas-v4','TPXO9-atlas-v5','TPXO9.1',
    'TPXO8-atlas','TPXO7.2','AODTM-5','AOTIM-5','AOTIM-5-2018',
    'Gr1km-v2','GOT4.7','GOT4.8','GOT4.10','FES2014','EOT20']
modelDropdown = widgets.Dropdown(
    options=model_list,
    value='GOT4.10',
    description='Model:',
    disabled=False,
)

# dropdown menu for setting ATLAS format model
atlas_list = ['OTIS','netcdf']
atlasDropdown = widgets.Dropdown(
    options=atlas_list,
    value='netcdf',
    description='ATLAS:',
    disabled=False,
)

# checkbox for setting if tide files are compressed
compressCheckBox = widgets.Checkbox(
    value=True,
    description='Compressed?',
    disabled=False,
)

# date picker widget for setting time
datepick = widgets.DatePicker(
    description='Date:',
    value = datetime.date.today(),
    disabled=False
)

# display widgets for setting directory and model
widgets.VBox([
    dirText,
    modelDropdown,
    atlasDropdown,
    compressCheckBox,
    datepick
])

In [ ]:
# default coordinates to use
LAT,LON = (32.86710263,-117.25750387)
m = leaflet.Map(center=(LAT,LON), zoom=12, basemap=leaflet.basemaps.Esri.WorldTopoMap)
# add control for zoom
zoom_slider = widgets.IntSlider(description='Zoom level:', min=0, max=15, value=7)
widgets.jslink((zoom_slider, 'value'), (m, 'zoom'))
zoom_control =  leaflet.WidgetControl(widget=zoom_slider, position='topright')
m.add_control(zoom_control)
# add marker with default location
marker = leaflet.Marker(location=(LAT,LON), draggable=True)
m.add_layer(marker)
# add text with marker location
markerText = widgets.Text(
    value='{0:0.8f},{1:0.8f}'.format(LAT,LON),
    description='Lat/Lon:',
    disabled=False
)

# add function for setting marker text if location changed
def set_marker_text(sender):
    LAT,LON = marker.location
    markerText.value = '{0:0.8f},{1:0.8f}'.format(LAT,wrap_longitudes(LON))
# add function for setting map center if location changed
def set_map_center(sender):
    m.center = marker.location
# add function for setting marker location if text changed
def set_marker_location(sender):
    LAT,LON = [float(i) for i in markerText.value.split(',')]
    marker.location = (LAT,LON)
    
# watch marker widgets for changes
marker.observe(set_marker_text)
markerText.observe(set_marker_location)
m.observe(set_map_center)
# add control for marker location
marker_control =  leaflet.WidgetControl(widget=markerText, position='bottomright')
m.add_control(marker_control)
m

In [ ]:
# leaflet location
LAT,LON = marker.location
# verify longitudes
LON = wrap_longitudes(LON)

# convert from calendar date to days relative to Jan 1, 1992 (48622 MJD)
YMD = datepick.value
# calculate a weeks forecast every minute
minutes = np.arange(7*1440)
tide_time = pyTMD.time.convert_calendar_dates(YMD.year, YMD.month,
    YMD.day, minute=minutes)
hours = minutes/60.0
# delta time (TT - UT1) file
delta_file = pyTMD.utilities.get_data_path(['data','merged_deltat.data'])

# get model parameters
model = pyTMD.model(dirText.value, format=atlasDropdown.value,
    compressed=compressCheckBox.value).elevation(modelDropdown.value)

# read tidal constants and interpolate to leaflet points
if model.format in ('OTIS','ATLAS'):
    amp,ph,D,c = extract_tidal_constants(np.atleast_1d(LON),
        np.atleast_1d(LAT), model.grid_file, model.model_file,
        model.projection, TYPE=model.type, METHOD='spline',
        EXTRAPOLATE=True, GRID=model.format)
    DELTAT = np.zeros_like(tide_time)
elif (model.format == 'netcdf'):
    amp,ph,D,c = extract_netcdf_constants(np.atleast_1d(LON),
        np.atleast_1d(LAT), model.grid_file, model.model_file,
        TYPE=model.type, METHOD='spline', EXTRAPOLATE=True,
        SCALE=model.scale, GZIP=model.compressed)
    DELTAT = np.zeros_like(tide_time)
elif (model.format == 'GOT'):
    amp,ph,c = extract_GOT_constants(np.atleast_1d(LON),
        np.atleast_1d(LAT), model.model_file, METHOD='spline',
        EXTRAPOLATE=True, SCALE=model.scale,
        GZIP=model.compressed)
    # interpolate delta times from calendar dates to tide time
    DELTAT = calc_delta_time(delta_file, tide_time)
elif (model.format == 'FES'):
    amp,ph = extract_FES_constants(np.atleast_1d(LON),
        np.atleast_1d(LAT), model.model_file, TYPE=model.type,
        VERSION=model.version, METHOD='spline', EXTRAPOLATE=True,
        SCALE=model.scale, GZIP=model.compressed)
    c = model.constituents
    # interpolate delta times from calendar dates to tide time
    DELTAT = calc_delta_time(delta_file, tide_time)
    
# calculate complex phase in radians for Euler's
cph = -1j*ph*np.pi/180.0
# calculate constituent oscillation
hc = amp*np.exp(cph)

# convert time from MJD to days relative to Jan 1, 1992 (48622 MJD)
# predict tidal elevations at time 1 and infer minor corrections
TIDE = predict_tidal_ts(tide_time, hc, c,
    DELTAT=DELTAT, CORRECTIONS=model.format)
MINOR = infer_minor_corrections(tide_time, hc, c,
    DELTAT=DELTAT, CORRECTIONS=model.format)
TIDE.data[:] += MINOR.data[:]
# convert to centimeters
TIDE.data[:] *= 100.0

# differentiate to calculate high and low tides
diff = np.zeros_like(tide_time, dtype=np.float64)
# forward differentiation for starting point
diff[0] = TIDE.data[1] - TIDE.data[0]
# backward differentiation for end point
diff[-1] = TIDE.data[-1] - TIDE.data[-2]
# centered differentiation for all others
diff[1:-1] = (TIDE.data[2:] - TIDE.data[0:-2])/2.0
# indices of high and low tides
htindex, = np.nonzero((np.sign(diff[0:-1]) >= 0) & (np.sign(diff[1:]) < 0))
ltindex, = np.nonzero((np.sign(diff[0:-1]) <= 0) & (np.sign(diff[1:]) > 0))

# create plot with tidal displacements, high and low tides and dates
fig,ax1 = plt.subplots(num=1)
ax1.plot(hours,TIDE.data,'k')
ax1.plot(hours[htindex],TIDE.data[htindex],'r*')
ax1.plot(hours[ltindex],TIDE.data[ltindex],'b*')
for h in range(24,192,24):
    ax1.axvline(h,color='gray',lw=0.5,ls='dashed',dashes=(11,5))
ax1.set_xlim(0,7*24)
ax1.set_ylabel('{0} Tidal Displacement [cm]'.format(model.name))
args = (YMD.year,YMD.month,YMD.day)
ax1.set_xlabel('Time from {0:4d}-{1:02d}-{2:02d} UTC [Hours]'.format(*args))
ax1.set_title(u'{0:0.6f}\u00b0N {1:0.6f}\u00b0W'.format(LAT,LON))
fig.subplots_adjust(left=0.10,right=0.98,bottom=0.10,top=0.95)
plt.show()